In [1]:
import os
from sys import getsizeof

In [2]:
import cudf
import cupy
from tqdm import tqdm
import numpy as np
import gc
import xgboost as xgb
from utils import amex_metric_np
from pathlib import Path
import dask_cudf

cudf.__version__, xgb.__version__, dask_cudf.__version__

('23.12.01', '1.7.6', '23.12.01')

In [3]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)

/home/costin/miniconda3/envs/rapids-23.12_upgrade/lib/python3.10/site-packages/dask_cuda/utils.py:170: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(


# Please register kaggle and install kaggle API by:
- `pip install kaggle`
- complete [authentication](https://www.kaggle.com/docs/api)

In [4]:
PATH = 'raid/data/ml/kaggle/amex'
os. getcwd()

'/mnt/d/workspace/Master/proiect_dmcf/triton_amex'

In [ ]:
os.umask(0)
Path(PATH).mkdir(parents=True,exist_ok=True)

In [ ]:
cmd = f'kaggle datasets download -d raddar/amex-data-integer-dtypes-parquet-format -p {PATH}'

In [ ]:
os.system(cmd)

In [5]:
os.listdir(PATH)

['amex-data-integer-dtypes-parquet-format.zip',
 'test.parquet',
 'train.parquet',
 'train_labels.csv',
 'train_labels.csv.zip']

In [ ]:
cmd = f'cd {PATH} && unzip -o amex-data-integer-dtypes-parquet-format.zip'
os.system(cmd)

In [6]:
os.listdir(PATH)

['amex-data-integer-dtypes-parquet-format.zip',
 'test.parquet',
 'train.parquet',
 'train_labels.csv',
 'train_labels.csv.zip']

# Basic EDA

In [7]:
%%time
train_cudf = cudf.read_parquet(f'{PATH}/train.parquet')
# print(train_cudf.shape)
# train_cudf.head()

CPU times: user 3.17 s, sys: 1.16 s, total: 4.33 s
Wall time: 10.7 s


In [11]:
train = dask_cudf.from_cudf(train_cudf, npartitions=100)
print(train.shape)
train.head()

MemoryError: ignored

In [ ]:
%%time
count_df = train.groupby('customer_ID').size().to_frame('num_profiles')
count_df.head()

In [ ]:
count_df.num_profiles.max()

In [ ]:
%%time
train['S_2'] = cudf.to_datetime(train['S_2'])
train.head()

In [ ]:
train.S_2.min(), train.S_2.max()

## Download the training data labels

In [ ]:
cmd = f'kaggle competitions download -c amex-default-prediction -f train_labels.csv -p {PATH}/'
os.system(cmd)

In [ ]:
cmd = f'cd {PATH} && unzip train_labels.csv.zip'
os.system(cmd)

In [ ]:
%%time
# trainL_cudf = cudf.read_csv(f'{PATH}/train_labels.csv')
# print(trainL_cudf.shape)
# trainL_cudf.head()

In [ ]:
%%time
trainl = dask_cudf.read_csv(f'{PATH}/train_labels.csv')
print(trainl.shape)
trainl.head()

In [ ]:
trainl['target'].value_counts()

In [ ]:
print(getsizeof(train))
print(getsizeof(trainl))

In [ ]:
%%time
train = train.merge(trainl, on='customer_ID', how='left')
print(train.shape)
train.head()

In [ ]:
train.persist()
train.head()

In [ ]:
train['cid'], _ = train.customer_ID.factorize()

In [ ]:
mask = train['cid']%4 == 0
tr,va = train.loc[~mask],train.loc[mask]
print("Verify target distribution is consistent across tr and va")
print(tr['target'].mean(), va['target'].mean())